In [2]:
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

* 자신에게 맞는 chromedriver와 위치 설정 선행필요

In [ ]:
browser = webdriver.Chrome('./chromedriver.exe')
browser.get('https://www.bigkinds.or.kr/v2/news/index.do')

# 함수정의부

In [ ]:
def login(e_mail, password):
    browser.find_element_by_xpath("//button[@class='btn-login login-modal-btn login-area-before']").click()
    browser.find_element_by_xpath("//input[@id='login-user-id']").send_keys(e_mail)
    browser.find_element_by_xpath("//input[@id='login-user-password']").send_keys(password)
    browser.find_element_by_xpath("//button[@id='login-btn']").click()
    time.sleep(5)
    return

In [ ]:
def datasetting(start_date, end_date, search_word):
    browser.get('https://www.bigkinds.or.kr/v2/news/index.do')

    browser.find_element_by_xpath("//input[@id='total-search-key']").send_keys(search_word)

    browser.find_element_by_xpath("//div[@class='tab-btn-inner tab1']").click()

    start_date = browser.find_element_by_xpath("//input[@id='search-begin-date']")
    start_date.click()
    start_date.send_keys(Keys.CONTROL + "a")
    start_date.send_keys(start_date)

    end_date = browser.find_element_by_xpath("//input[@id='search-end-date']")
    end_date.click()
    end_date.send_keys(Keys.CONTROL + "a")
    end_date.send_keys(end_date)

    time.sleep(1)
    browser.find_element_by_xpath("//button[@class='btn btn-search news-search-btn news-report-search-btn']").click()

    WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.collap-wrp > div.collap-head > button#collapse-step-3")))
    browser.find_element_by_css_selector('div.collap-wrp > div.collap-head > button#collapse-step-3').click()
    WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.CLASS_NAME, "btn btn-wh ml-auto news-download-btn mobile-excel-download")))
    browser.find_element_by_xpath("//button[@class='btn btn-wh ml-auto news-download-btn mobile-excel-download']").click()

    return 

* 미흡한부분 : [엑셀다운로드]를 클릭하면 다운로드 위치에 저장되는데. 이걸 지정해서 저장된 엑셀파일을 불러오는 코드 만들면 아래 전처리 내용을 바로 하도록 코딩해서 신문기사를 주기적으로 갱신되도록 할 수 있을 것 같음

# 실행

* A조의 경우 일정시간마다 함수를 콜백해서 데이터를 갱신하는 내용을 추가하도록 했다고함

In [ ]:
# 파일 갱신
login('Guardians555@gmail.com', 'Guardians555')
datasetting('2021-01-01', '2021-09-25', '온실가스')

# 데이터 전처리

* 사진만 있는 기사, 짧은 정보성 기사는 의미없다고 판단해서 먼저 제목에 '온실가스'포함 그리고  keyword의 len가 가장 높은 순서대로 내림차순하여 5개 기사 선정

In [46]:
df = pd.read_excel('./list.xlsx', names=['identical', 'date', 'press', 'name', 'title', 'c1', 'c2', 'c3', 'a1', 'a2', 'a3', 'person', 'place', 'institute', 'keyword', 'topkeyword', 'body', 'url', 'tf'])
df = df[df.title.str.contains('온실가스')]
dff = df[['date', 'press', 'title', 'keyword', 'url']]
dff.head(5)

,date,press,title,keyword,url
7,20210926,서울경제,"탄소 중립 첫 발 BGF리테일, 온실가스 인벤토리 국제 인증서 획득","탄소,중립,BGF리테일,획득,온실,가스,인벤토리,국제,인증서,BGF리테일,물류,전문...",http://www.sedaily.com/NewsView/22RM56HXVV
11,20210926,아시아경제,"BGF리테일 ""온실가스 인벤토리 구축 탄소중립 첫 발""","BGF,리테일,온실,가스,인벤토리,구축,탄소중립,BGF리테일,자회사,물류,전문,BG...",https://www.asiae.co.kr/article/20210926120949...
17,20210926,디지털타임스,"BGF리테일, 온실가스 관리 시스템 구축한다","BGF리테일,온실,가스,관리,구축,BGF리테일,통합,온실,가스,인벤토리,구축,검증,...",http://www.dt.co.kr/contents.html?article_no=2...
33,20210924,KBS,"기후위기비상행동, “도, 2030년까지 온실가스 50% 줄여야”","기후위기비상행동,2030년,온실,가스,50%,기후,위기,전북,비상행동,정의당,전북도...",https://news.kbs.co.kr/news/view.do?ncd=528656...
34,20210924,KBS,"정의당 전남도당, ‘온실가스 배출 1위’ 포스코 규탄","정의당,전남도당,1위,온실,가스,배출,포스코,규탄,정의당,전남도당,소본부,포스코,광...",https://news.kbs.co.kr/news/view.do?ncd=528655...


In [47]:
target = list(zip(dff.index.tolist(), dff.keyword.astype(str).tolist()))
target.sort(key = lambda x:len(x[1]), reverse=True)

In [48]:
index = []
for i in target[0:5]:
    index.append(i[0])

In [49]:
df_target = dff.loc[index]
df_target

,date,press,title,keyword,url
1676,20210825,한겨레,“신규 석탄발전소 7곳 가동되면 온실가스 감축 노력 물거품 돼”,"신규,석탄,발전소,가동,온실,가스,감축,노력,물거품,기후솔루션,단체,이름,운동,거리...",http://www.hani.co.kr/arti/society/environment...
1854,20210823,한겨레,“온실가스 65만톤 감축” 산림청 홍보 뒤엔 숲 37% 파괴 있었다,"온실,가스,감축,산림청,홍보,파괴,37%,한국,산림청,레드플러스,REDD,시범사업,...",http://www.hani.co.kr/arti/society/environment...
1093,20210903,머니투데이,국제사회에 대한 책임 ‘온실가스 감축’,"국제사회,감축,책임,온실,가스,세상,기후위기,기본법,대응,탄소,중립,기업,환경,단체...",http://news.moneytoday.co.kr/view/mtview.php?n...
3039,20210802,내일신문,[개발사업 20년과 환경정책│② 이명박정부~문재인정부] 영주댐은 '만수위' 온실가스...,"사업,환경정책,이명박정부,문재인정부,영주댐,만수위,온실,가스,감축,장마,시작,연일,...",http://www.naeil.com/news_view/?id_art=394437
6627,20210530,서울경제,"[전문] 文대통령 ""온실가스 추가 감축""...국제사회 재천명","대통령,감축,온실,가스,추가,국제사회,재천명,P4G,서울,녹색,미래,정상,회의,개회...",http://www.sedaily.com/NewsView/22MK9JVN6A


In [50]:
df_target.url.tolist()

['http://www.hani.co.kr/arti/society/environment/1009003.html',
 'http://www.hani.co.kr/arti/society/environment/1008681.html',
 'http://news.moneytoday.co.kr/view/mtview.php?no=2021090117077846949&type=2',
 'http://www.naeil.com/news_view/?id_art=394437',
 'http://www.sedaily.com/NewsView/22MK9JVN6A']